# Recommender Notebook


### Imports 
---

In [41]:
import pandas as pd
import numpy as np
from scipy import sparse

from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

%matplotlib inline

### Data and Cleaning
---

Load in our user ratings data set and vegetable info data set.

In [42]:
ratings = pd.read_csv('./data/veggie_ratings.csv')
ratings.head()

,Timestamp,Artichokes,Arugula,Asparagus,Bush beans (green beans),Pole beans(green beans),Beets,Bok Choy,Broccoli,Brussel Sprouts,...,Radishes,Rhubarb,Rutabaga,Shallots,Spinach,Summer Squash,Winter Squash,Swiss Chard,Tomatoes,Turnips
0,10/12/2021 17:47:21,5,5,5,4,4,1,3,5,5,...,1,2,2,4,5,1,1,3,5,2
1,10/12/2021 17:57:50,2,4,2,4,2,1,1,3,1,...,1,1,1,3,5,1,1,1,4,1
2,10/12/2021 18:17:26,3,3,5,3,3,1,1,4,3,...,1,2,1,5,5,1,1,1,4,1
3,10/12/2021 18:18:15,4,2,1,2,2,2,3,3,3,...,3,2,2,2,2,2,2,2,1,2
4,10/12/2021 18:21:26,3,2,5,5,5,1,1,5,1,...,1,1,1,1,1,1,1,1,5,1


In [65]:
veg_info = pd.read_csv('./data/veg_info.csv')
veg_info.head()

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use,link
0,Artichokes,full sunlight,well drained,high,"6, 7, 8, 9",perennial,3.0,3 to 6 feet,2 to 4 feet,85 - 100 days,violet,light green,spreading mass/upright,high,Not native to North America,none,edible landscaping,https://www.almanac.com/plant/artichokes
1,Arugula,full sunlight or partial shade,low fertility,medium,"3, 4, 5, 6, 7, 8, 9, 10, 11",annual,1.0,1 to 2 feet,0.5 to 1 foot,4 - 7 weeks,white,medium green,"cushion, mound, upright",medium,Not native to North America,none,edible flowers,https://www.almanac.com/plant/arugula
2,Asparagus,full sunlight or partial shade,well drained,low,"2, 3, 4, 5, 6, 7, 8",perennial,2.0,5 to 9 feet,2 to 2.5 feet,2 - 3 years,green,light green,upright,low,Not native to North America,"Bears ornamental fruit - small, bright red ber...",edible landscaping,https://www.almanac.com/plant/asparagus
3,Bush beans (green beans),full,well drained,medium,"3, 4, 5, 6, 7, 8, 9, 10",annual,1.0,1 to 3 feet,1 to 2 feet,50 - 60 days,white,medium green,spreading mass,low,Not native to North America but cultivated wor...,Bears ornamental fruit - some varieties have p...,edible landscaping,https://www.almanac.com/plant/beans
4,Beets,full sunlight or partial shade,low fertility,consistent,"2, 3, 4, 5, 6, 7, 8, 9, 10",annual,1.0,3 to 5 inches,0.5 to 1 foot,7 - 8 weeks,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - native to Europe...,none,edible landscaping,https://www.almanac.com/plant/beets


As is always good practice I checked for nulls, even though I don't expect any, and also checked out dtypes. All looks good.

In [43]:
ratings.isnull().sum()

Timestamp                   0
Artichokes                  0
Arugula                     0
Asparagus                   0
Bush beans (green beans)    0
Pole beans(green beans)     0
Beets                       0
Bok Choy                    0
Broccoli                    0
Brussel Sprouts             0
Cabbage                     0
Carrots                     0
Cauliflower                 0
Celery                      0
Chives                      0
Collards                    0
Corn                        0
Cucumbers                   0
Eggplant                    0
Garlic                      0
Kale                        0
Leeks                       0
Lettuce                     0
Mustard Greens              0
Onions                      0
Parsnips                    0
Peas                        0
Peppers                     0
Potatoes                    0
Pumpkin                     0
Radishes                    0
Rhubarb                     0
Rutabaga                    0
Shallots  

In [44]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Timestamp                 134 non-null    object
 1   Artichokes                134 non-null    int64 
 2   Arugula                   134 non-null    int64 
 3   Asparagus                 134 non-null    int64 
 4   Bush beans (green beans)  134 non-null    int64 
 5   Pole beans(green beans)   134 non-null    int64 
 6   Beets                     134 non-null    int64 
 7   Bok Choy                  134 non-null    int64 
 8   Broccoli                  134 non-null    int64 
 9   Brussel Sprouts           134 non-null    int64 
 10  Cabbage                   134 non-null    int64 
 11  Carrots                   134 non-null    int64 
 12  Cauliflower               134 non-null    int64 
 13  Celery                    134 non-null    int64 
 14  Chives                    

In [81]:
veg_info.isnull().sum()

plant               5
sunlight            5
soil_condition      5
moisture_level      5
hardiness_zones     5
lifecycle           5
difficulty          5
height              5
spread              5
time_to_mature      5
flower_color       25
foliage_color       5
shape               5
frost_tolerance     5
special_att1        5
special_att2        5
special_use         5
link                5
dtype: int64

Not positive why it's claiming 5 nulls across the board.  ``

In [82]:
veg_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   plant            37 non-null     object 
 1   sunlight         37 non-null     object 
 2   soil_condition   37 non-null     object 
 3   moisture_level   37 non-null     object 
 4   hardiness_zones  37 non-null     object 
 5   lifecycle        37 non-null     object 
 6   difficulty       37 non-null     float64
 7   height           37 non-null     object 
 8   spread           37 non-null     object 
 9   time_to_mature   37 non-null     object 
 10  flower_color     17 non-null     object 
 11  foliage_color    37 non-null     object 
 12  shape            37 non-null     object 
 13  frost_tolerance  37 non-null     object 
 14  special_att1     37 non-null     object 
 15  special_att2     37 non-null     object 
 16  special_use      37 non-null     object 
 17  link             3

Remove the timestamp column and replace it with a numerical index as well as drop a feature I no longer want to include (pole beans - too similar to bush beans).

In [45]:
ratings['user_id'] = ratings.index
ratings.drop(columns=['Timestamp', 'Pole beans(green beans)'], inplace=True)

The Google Sheets format for the survey data is in the opposite layout I wanted for our table so I used Panda's `.melt()` function to "unpivot" our data.

In [48]:
ratings = ratings.melt(id_vars = 'user_id')

Check the shape and run a `.head()` to check our dimensions and layout, looks good!

In [49]:
ratings.shape

(5092, 3)

In [50]:
ratings.head()

,user_id,variable,value
0,0,Artichokes,5
1,1,Artichokes,2
2,2,Artichokes,3
3,3,Artichokes,4
4,4,Artichokes,3


Do a quick column rename for readability's sake.

In [53]:
ratings = ratings.rename(columns={'variable': 'veggie_name', 'value': 'rating'})

In [54]:
ratings.head()

,user_id,veggie_name,rating
0,0,Artichokes,5
1,1,Artichokes,2
2,2,Artichokes,3
3,3,Artichokes,4
4,4,Artichokes,3


### Data Processing
---

Pivot our table so vegetables are the index and columns are our user_ids

In [55]:
pivot = pd.pivot_table(ratings, index='veggie_name', columns='user_id', values='rating')
pivot.head()

user_id,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
veggie_name,,,,,,,,,,,,,,,,,,,,,
Artichokes,5,2,3,4,3,4,4,4,5,2,...,3,3,2,3,5,5,1,3,4,2
Arugula,5,4,3,2,2,1,2,5,5,3,...,4,3,3,4,1,3,2,3,1,2
Asparagus,5,2,5,1,5,4,4,5,2,5,...,5,5,4,5,5,3,2,4,3,5
Beets,1,1,1,2,1,4,1,4,5,1,...,2,3,1,5,1,1,2,1,2,5
Bok Choy,3,1,1,3,1,4,4,4,1,5,...,3,3,1,4,2,2,2,4,1,4


Apply the scipy `.csr_matrix()` function to our pivoted table before we get our cosine distances.

In [56]:
sparse_pivot = sparse.csr_matrix(pivot)

Calculate the cosine distances between our x and y variables and save it to a new variable.

In [57]:
dists = cosine_distances(sparse_pivot)
dists

array([[0.        , 0.10662537, 0.09564753, ..., 0.10744898, 0.14118151,
        0.11712129],
       [0.10662537, 0.        , 0.07184178, ..., 0.08738546, 0.14670153,
        0.12323257],
       [0.09564753, 0.07184178, 0.        , ..., 0.06230751, 0.14093961,
        0.09054579],
       ...,
       [0.10744898, 0.08738546, 0.06230751, ..., 0.        , 0.13880848,
        0.0890425 ],
       [0.14118151, 0.14670153, 0.14093961, ..., 0.13880848, 0.        ,
        0.11349265],
       [0.11712129, 0.12323257, 0.09054579, ..., 0.0890425 , 0.11349265,
        0.        ]])

Save these distances to a dataframe with vegetable names as the index and columns.

In [60]:
recommender_df = pd.DataFrame(dists, columns=pivot.index, index=pivot.index)
recommender_df.head()

veggie_name,Artichokes,Arugula,Asparagus,Beets,Bok Choy,Broccoli,Brussel Sprouts,Bush beans (green beans),Cabbage,Carrots,...,Radishes,Rhubarb,Rutabaga,Shallots,Spinach,Summer Squash,Swiss Chard,Tomatoes,Turnips,Winter Squash
veggie_name,,,,,,,,,,,,,,,,,,,,,
Artichokes,0.000000,0.106625,0.095648,0.165555,0.118225,0.097136,0.107318,0.121322,0.113323,0.113787,...,0.126521,0.169771,0.123633,0.096574,0.094731,0.113701,0.117724,0.107449,0.141182,0.117121
Arugula,0.106625,0.000000,0.071842,0.147518,0.132488,0.083784,0.090161,0.095887,0.103993,0.092288,...,0.119242,0.130691,0.137964,0.087220,0.067948,0.111108,0.118808,0.087385,0.146702,0.123233
Asparagus,0.095648,0.071842,0.000000,0.160414,0.107279,0.047656,0.074525,0.060137,0.079576,0.058942,...,0.114820,0.131626,0.116850,0.086356,0.040759,0.076392,0.128745,0.062308,0.140940,0.090546
Beets,0.165555,0.147518,0.160414,0.000000,0.147149,0.177839,0.155912,0.159939,0.157683,0.174046,...,0.110128,0.178732,0.142540,0.150989,0.155327,0.185653,0.151678,0.161731,0.136879,0.181784
Bok Choy,0.118225,0.132488,0.107279,0.147149,0.000000,0.101795,0.120795,0.122341,0.113047,0.119672,...,0.128547,0.140406,0.089950,0.101320,0.108681,0.105129,0.105006,0.130676,0.126066,0.096839


Save our recommender data frame for use in our app later.

In [61]:
recommender_df.to_csv('./data/recommender_df.csv')

Next do a quick test run of our recommender.  Beets, bears, Battlestar Galactica.

In [62]:
recommender_df['Beets'].sort_values()[1:11]

veggie_name
Radishes       0.110128
Turnips        0.136879
Rutabaga       0.142540
Bok Choy       0.147149
Arugula        0.147518
Shallots       0.150989
Swiss Chard    0.151678
Leeks          0.152880
Peppers        0.152989
Potatoes       0.153980
Name: Beets, dtype: float64

In [67]:
veg_lst = recommender_df[['Beets']].sort_values(by='Beets')[1:11]
veg_lst

veggie_name,Beets
veggie_name,
Radishes,0.110128
Turnips,0.136879
Rutabaga,0.142540
Bok Choy,0.147149
Arugula,0.147518
Shallots,0.150989
Swiss Chard,0.151678
Leeks,0.152880
Peppers,0.152989


Create a list of recommended vegetables to parse our veg_info table for

In [68]:
veg_lst = veg_lst.index.tolist()
veg_lst

['Radishes',
 'Turnips',
 'Rutabaga',
 'Bok Choy',
 'Arugula',
 'Shallots',
 'Swiss Chard',
 'Leeks',
 'Peppers',
 'Potatoes']

Iterate through veg_info and print out the rows which match our veggies from the veg_list

In [69]:
for num in range(0, veg_info.shape[0]):
    for veg in veg_lst:
            if veg_info['plant'][num] == veg:
                print(veg_info.loc[[num]])

     plant                        sunlight soil_condition moisture_level  \
1  Arugula  full sunlight or partial shade  low fertility         medium   

               hardiness_zones lifecycle  difficulty       height  \
1  3, 4, 5, 6, 7, 8, 9, 10, 11    annual         1.0  1 to 2 feet   

          spread time_to_mature flower_color foliage_color  \
1  0.5 to 1 foot    4 - 7 weeks        white  medium green   

                     shape frost_tolerance                 special_att1  \
1  cushion, mound, upright          medium  Not native to North America   

  special_att2     special_use                                   link  
1         none  edible flowers  https://www.almanac.com/plant/arugula  
      plant                        sunlight soil_condition moisture_level  \
5  Bok Choy  full sunlight or partial shade   well drained     consistent   

                  hardiness_zones lifecycle  difficulty       height  \
5  2, 3, 4, 5, 6, 7, 8, 9, 10, 11    annual         2.0  1 to

Wrote a function to get top 10 recommended vegetables and then pull their information from our veg_info dataframe.

In [70]:
def pull_veggie(vegetable):
    '''
    This function serves to: 
    1) Pull the top 10 recommended vegetables from our recommender df
    2) Pull the data from our veg_info dataframe with specific information for those 10 recommended veggies
    Note: input needs to be a string from with a capitalized first letter
    '''
    veg_lst = recommender_df[[vegetable]].sort_values(by=vegetable)[1:11]
    veg_lst = veg_lst.index.tolist()
    
    created_df = pd.DataFrame(columns=veg_info.columns)
    
    for num in range(0, veg_info.shape[0]):
        for veg in veg_lst:
                if veg_info['plant'][num] == veg:
                    created_df = created_df.append(veg_info.loc[[num]])
                    
    return created_df

In [71]:
pull_veggie('Beets')

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use,link
1,Arugula,full sunlight or partial shade,low fertility,medium,"3, 4, 5, 6, 7, 8, 9, 10, 11",annual,1.0,1 to 2 feet,0.5 to 1 foot,4 - 7 weeks,white,medium green,"cushion, mound, upright",medium,Not native to North America,none,edible flowers,https://www.almanac.com/plant/arugula
5,Bok Choy,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9, 10, 11",annual,2.0,1 to 2 feet,1 to 1.5 feet,45 days,NaN,medium green,"cushion, mound, or clump",low,Not native to North America - not known in the...,none,edible landscaping,https://www.gardeningknowhow.com/edible/vegeta...
19,Leeks,full sunlight or partial shade,well drained and high fertility,consistent,"7, 8",annual,2.0,2 to 3 feet,0.5 to 1 foot,120 - 150 days,NaN,medium and dark green,upright/fan-shaped,medium (more tolerant than other onions),Not native to North America - cultivated in Eg...,none,edible landscaping,https://www.almanac.com/video/growing-leeks-seed
25,Peppers,full,well drained,consistent,"9, 10, 11",annual,2.0,1 to 3 feet,1 to 3 feet,60 - 90 days,white,dark green,upright,low,Not native to North America - native to tropic...,Bears ornamental fruit - green bell peppers tu...,edible landscaping,https://www.almanac.com/plant/bell-peppers
26,Potatoes,full,high acid and well drained,consistent,"3, 4, 5, 6, 7, 8, 9, 10",annual,1.0,1.5 to 3 feet,1.5 to 3 feet,80 - 100 days,violet,medium green,"cushion, mound, or clump",medium,Not native to North America - native to the An...,none,none,https://www.almanac.com/plant/potatoes
28,Radishes,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9, 10",annual,1.0,0.5 to 1.5 feet,0.5 to 0.75 feet,35 - 45 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - Mediterranean or...,none,none,https://www.almanac.com/plant/radishes
31,Shallots,full sunlight or partial shade,well drained,consistent,"4, 5, 6, 7, 8, 9, 10",perennial,2.0,1 to 2 feet,0.5 to 1 foot,90 days,NaN,medium green,upright,medium,Not native to North America - Middle East origin,none,edible landscaping,https://www.almanac.com/growing-shallots-how-p...
34,Swiss Chard,full sunlight or partial shade,well drained,consistent,"6, 7, 8, 9, 10",annual,1.0,1 to 3 feet,0.5 to 2 feet,50 - 60 days,NaN,"medium/dark green, red, purple, and yellow","cushion, mound, or clump, and upright",medium,Not native to North America - domestivated fro...,none,edible landscaping,https://www.almanac.com/plant/swiss-chard
36,Turnips,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9",annual,1.0,1 to 1.5 feet,6 to 8 inches,40 - 55 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - not known in the...,none,none,https://www.almanac.com/plant/turnips


Additional input, skill_level = user's gardeneing skill level, 

In [72]:
def pull_veggie_v2(vegetable, skill_level):
    '''
    This function serves to: 
    1) Pull the top 10 recommended vegetables from our recommender df
    2) Pull the data from our veg_info dataframe with specific information for those 10 recommended veggies
    Note: input needs to be a string from with a capitalized first letter
    '''
    veg_lst = recommender_df[[vegetable]].sort_values(by=vegetable)[1:11]
    veg_lst = veg_lst.index.tolist()
    
    created_df = pd.DataFrame(columns=veg_info.columns)
    
    for num in range(0, veg_info.shape[0]):
        for veg in veg_lst:
                if veg_info['plant'][num] == veg and veg_info['difficulty'][num] <= skill_level:
                    created_df = created_df.append(veg_info.loc[[num]])
                    
    return created_df

In [73]:
pull_veggie_v2('Beets', 1)

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use,link
1,Arugula,full sunlight or partial shade,low fertility,medium,"3, 4, 5, 6, 7, 8, 9, 10, 11",annual,1.0,1 to 2 feet,0.5 to 1 foot,4 - 7 weeks,white,medium green,"cushion, mound, upright",medium,Not native to North America,none,edible flowers,https://www.almanac.com/plant/arugula
26,Potatoes,full,high acid and well drained,consistent,"3, 4, 5, 6, 7, 8, 9, 10",annual,1.0,1.5 to 3 feet,1.5 to 3 feet,80 - 100 days,violet,medium green,"cushion, mound, or clump",medium,Not native to North America - native to the An...,none,none,https://www.almanac.com/plant/potatoes
28,Radishes,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9, 10",annual,1.0,0.5 to 1.5 feet,0.5 to 0.75 feet,35 - 45 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - Mediterranean or...,none,none,https://www.almanac.com/plant/radishes
34,Swiss Chard,full sunlight or partial shade,well drained,consistent,"6, 7, 8, 9, 10",annual,1.0,1 to 3 feet,0.5 to 2 feet,50 - 60 days,NaN,"medium/dark green, red, purple, and yellow","cushion, mound, or clump, and upright",medium,Not native to North America - domestivated fro...,none,edible landscaping,https://www.almanac.com/plant/swiss-chard
36,Turnips,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9",annual,1.0,1 to 1.5 feet,6 to 8 inches,40 - 55 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - not known in the...,none,none,https://www.almanac.com/plant/turnips


Additional input, hardiness_zone = user's hardiness zone.

NOTE: I kept the hardiness_zone data as a string so I could use `in` to see if the user's specified zone was in the data

In [74]:
def pull_veggie_v3(vegetable, skill_level, hardiness_zone):
    '''
    This function serves to: 
    1) Pull the top 10 recommended vegetables from our recommender df
    2) Pull the data from our veg_info dataframe with specific information for those 10 recommended veggies
    3) Check if the difficuly level is <= the user's specificed level
    4) Check if the user's hardiness zone is in the top 10 veggie choices
    Note: input needs to be a string from with a capitalized first letter 
    '''
    veg_lst = recommender_df[[vegetable]].sort_values(by=vegetable)[1:11]
    veg_lst = veg_lst.index.tolist()
    
    created_df = pd.DataFrame(columns=veg_info.columns)
    
    for num in range(0, veg_info.shape[0]):
        for veg in veg_lst:
                if veg_info['plant'][num] == veg and veg_info['difficulty'][num] <= skill_level and (hardiness_zone in veg_info['hardiness_zones'][num]):
                    created_df = created_df.append(veg_info.loc[[num]])
                    
    return created_df

In [75]:
pull_veggie_v3('Beets', 1, '4')

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use,link
1,Arugula,full sunlight or partial shade,low fertility,medium,"3, 4, 5, 6, 7, 8, 9, 10, 11",annual,1.0,1 to 2 feet,0.5 to 1 foot,4 - 7 weeks,white,medium green,"cushion, mound, upright",medium,Not native to North America,none,edible flowers,https://www.almanac.com/plant/arugula
26,Potatoes,full,high acid and well drained,consistent,"3, 4, 5, 6, 7, 8, 9, 10",annual,1.0,1.5 to 3 feet,1.5 to 3 feet,80 - 100 days,violet,medium green,"cushion, mound, or clump",medium,Not native to North America - native to the An...,none,none,https://www.almanac.com/plant/potatoes
28,Radishes,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9, 10",annual,1.0,0.5 to 1.5 feet,0.5 to 0.75 feet,35 - 45 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - Mediterranean or...,none,none,https://www.almanac.com/plant/radishes
36,Turnips,full sunlight or partial shade,well drained,consistent,"2, 3, 4, 5, 6, 7, 8, 9",annual,1.0,1 to 1.5 feet,6 to 8 inches,40 - 55 days,NaN,medium green,"cushion, mound, or clump",medium,Not native to North America - not known in the...,none,none,https://www.almanac.com/plant/turnips


In [76]:
veg_info.head(1)

,plant,sunlight,soil_condition,moisture_level,hardiness_zones,lifecycle,difficulty,height,spread,time_to_mature,flower_color,foliage_color,shape,frost_tolerance,special_att1,special_att2,special_use,link
0,Artichokes,full sunlight,well drained,high,"6, 7, 8, 9",perennial,3.0,3 to 6 feet,2 to 4 feet,85 - 100 days,violet,light green,spreading mass/upright,high,Not native to North America,none,edible landscaping,https://www.almanac.com/plant/artichokes


In [77]:
#choose additional output data

def pull_veggie_v6(vegetable, skill_level, hardiness_zone):
    '''
    This function serves to: 
    1) Pull the top 10 recommended vegetables from our recommender df
    2) Pull the data from our veg_info dataframe with specific information for those 10 recommended veggies
    3) Check if the difficuly level is <= the user's specificed level
    4) Check if the user's hardiness zone is in the top 10 veggie choices
    5) Print some important facts about the plant
    Note: vegetable = string, skill_level = int, hardiness_zone = int as a string 
    '''
    veg_lst = recommender_df[[vegetable]].sort_values(by=vegetable)[1:11]
    veg_lst = veg_lst.index.tolist()
    
    created_df = pd.DataFrame(columns=veg_info.columns)
    
    for num in range(0, veg_info.shape[0]):
        for veg in veg_lst:
                if veg_info['plant'][num] == veg and veg_info['difficulty'][num] <= skill_level and (hardiness_zone in veg_info['hardiness_zones'][num]):
                    created_df = created_df.append(veg_info.loc[[num]])
    
    if created_df.shape[0] == 0:
        return "Unfortunately it doesn't look like we found any great matches for you, try adjusting your parameters"
    
    print("Our top picks for you are...")
    print()
    for num in range(0, created_df.shape[0]): 
        print(f"#{num+1} - {veg_lst[num]}!")
        print()
        print(f"A few important things to know before you grow {veg_lst[num]}:")
        print()
        print(f"1) Optimal sun exposure: {created_df.iloc[num][1]}")
        print()
        print(f"2) Ideal soil conditions: {created_df.iloc[num][2]}")
        print()
        print(f"3) Moisture level: {created_df.iloc[num][3]}")
        print()
        print(f"4) Expected spread: {created_df.iloc[num][7]}")
        print()
        print(f"5) Expected height: {created_df.iloc[num][8]}")
        print()
        print(f"6) Estimated time to harvest: {created_df.iloc[num][9]}")
        print()
        print("*" * 45)
        print()

In [78]:
pull_veggie_v6('Beets', 2, '4')

Our top picks for you are...

#1 - Radishes!

A few important things to know before you grow Radishes:

1) Optimal sun exposure: full sunlight or partial shade

2) Ideal soil conditions: low fertility

3) Moisture level: medium

4) Expected spread: 1 to 2 feet

5) Expected height: 0.5 to 1 foot

6) Estimated time to harvest: 4 - 7 weeks

*********************************************

#2 - Turnips!

A few important things to know before you grow Turnips:

1) Optimal sun exposure: full sunlight or partial shade

2) Ideal soil conditions: well drained

3) Moisture level: consistent

4) Expected spread: 1 to 2 feet

5) Expected height: 1 to 1.5 feet

6) Estimated time to harvest: 45 days

*********************************************

#3 - Rutabaga!

A few important things to know before you grow Rutabaga:

1) Optimal sun exposure: full

2) Ideal soil conditions: high acid and well drained

3) Moisture level: consistent

4) Expected spread: 1.5 to 3 feet

5) Expected height: 1.5 to 3 feet


In [79]:
veg_info.iloc[1][4]

'3, 4, 5, 6, 7, 8, 9, 10, 11'